In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from collections import  defaultdict

# preprocessing 
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from yellowbrick.classifier import ConfusionMatrix
from sklearn.metrics import classification_report, accuracy_score

# models
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import (
    RandomForestClassifier,
    HistGradientBoostingClassifier
)

# scripts
from src.wrapped import Wrapped
from src.analysesV02 import Analytics
from src.trainV02 import TrainModels 

# Importar dados

In [2]:
tm = TrainModels()
ac = Analytics()
wp = Wrapped(
    '../data/row/',
    '../data/processed/',
    '../data/files/'
)

In [3]:
# dataset
df = wp.load_data('df_instrumentos_features_selecionadas').drop(columns=['file_name'])
df.columns

Index(['tonnetz0', 'tonnetz1', 'chroma1', 'chroma2', 'chroma3', 'chroma4',
       'chroma5', 'chroma6', 'chroma7', 'mfcc0', 'mfcc1', 'mfcc2', 'mfcc3',
       'mfcc4', 'mfcc5', 'mfcc6', 'mfcc7', 'mfcc8', 'mfcc9', 'mfcc10',
       'mfcc11', 'mfcc12', 'mfcc13', 'mfcc14', 'mfcc15', 'mfcc16', 'mfcc17',
       'mfcc18', 'mfcc19', 'chroma8', 'chroma9', 'chroma10', 'chroma11',
       'chroma12', 'spectral_band2', 'spectral_band3', 'spectral_band4',
       'spectral_centroid', 'spectral_onset', 'spectral_rolloff', 'spectogram',
       'mel_spectogram', 'zero_crossing_rate', 'zero_crossing', 'instrumento'],
      dtype='object')

# Pré Processamento

#### instrumentos por classe
- corda: cello, guitar, violin, bass, banjo, mandolin e ukulele
- percusao: mallet_percussion, drums e cymbals
- sopro: clarinet, trombone, flute, trumpet, saxophone
- aerofone: accordion, organ, piano 

In [4]:
dict_cat_inst = {
    'aerofone': ["accordion", "organ", "piano"],
    'percusao': ["mallet_percussion", "drums", "cymbals"],
    'sopro': ["clarinet", "trombone", "flute", "trumpet", "saxophone"],
    'corda': ["cello", "guitar", "violin", "bass", "banjo", "mandolin", "ukulele"]
}

In [5]:
df_aerofone = df[df['instrumento'].isin(dict_cat_inst['aerofone'])]
df_aerofone.insert(0, 'cat_inst', 'aerofone')

df_percusao = df[df['instrumento'].isin(dict_cat_inst['percusao'])]
df_percusao.insert(0, 'cat_inst', 'percusao')

df_sopro = df[df['instrumento'].isin(dict_cat_inst['sopro'])]
df_sopro.insert(0, 'cat_inst', 'sopro')

df_corda = df[df['instrumento'].isin(dict_cat_inst['corda'])]
df_corda.insert(0, 'cat_inst', 'corda')

df_cat_inst = pd.concat([df_aerofone, df_percusao, df_sopro, df_corda])
df_cat_inst

,cat_inst,tonnetz0,tonnetz1,chroma1,chroma2,chroma3,chroma4,chroma5,chroma6,chroma7,...,spectral_band3,spectral_band4,spectral_centroid,spectral_onset,spectral_rolloff,spectogram,mel_spectogram,zero_crossing_rate,zero_crossing,instrumento
0,aerofone,0.010380,0.038218,0.513887,0.581963,0.434641,0.293949,0.317826,0.271148,0.548258,...,2949.581321,3409.640722,2600.055158,1.098243,5260.821321,-44.640682,-30.202150,0.149672,33095,accordion
1,aerofone,0.084643,-0.035337,0.175587,0.429345,0.840043,0.384524,0.282545,0.346341,0.482365,...,3038.064172,3516.595715,2385.271420,1.360742,5225.124166,-46.575737,-59.323132,0.112181,24785,accordion
2,aerofone,-0.030127,0.082404,0.453814,0.219808,0.228484,0.446050,0.178916,0.250981,0.327460,...,2735.690428,3256.491175,2069.850731,1.343197,4287.155824,-56.276707,-71.512909,0.096007,21230,accordion
3,aerofone,0.157274,-0.066099,0.209729,0.338755,0.660264,0.256625,0.275209,0.180417,0.399448,...,3446.105958,3855.791675,2903.979876,1.218963,6751.133809,-42.723160,-44.649315,0.144010,31836,accordion
4,aerofone,-0.049096,-0.029067,0.484400,0.334572,0.441629,0.679485,0.382958,0.326033,0.205775,...,2223.101638,2883.225952,1251.374287,1.339527,2263.034730,-62.227947,-78.196365,0.059804,13214,accordion
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8019,corda,0.051950,0.691044,0.773323,0.582712,0.532384,0.274606,0.127275,0.109617,0.158079,...,1392.144514,1906.686246,658.017342,1.227272,1367.932950,-35.302883,-68.164001,0.019043,4212,violin
8020,corda,0.015595,-0.058762,0.120028,0.238263,0.234643,0.267743,0.228591,0.260930,0.267262,...,2995.859927,3383.563150,3573.542978,1.354204,6552.813508,-43.596504,-73.498703,0.230540,50962,violin
8021,corda,0.093624,-0.031124,0.253249,0.321181,0.309254,0.265595,0.265015,0.155659,0.181344,...,2520.260771,3174.287104,1421.358194,1.173742,2751.578644,-48.512993,-66.277359,0.065123,14407,violin
8022,corda,0.004588,-0.009204,0.124913,0.174113,0.185558,0.169614,0.149416,0.075544,0.081894,...,2842.937048,3232.221409,3720.081212,1.332893,6207.732687,-37.720039,-72.552101,0.241853,53491,violin


In [10]:
print('Sem voice e synthesizer:', df.shape[0] - (df[df.instrumento == 'voice'].shape[0] + df[df.instrumento == 'synthesizer'].shape[0]))

Sem voice e synthesizer: 7649


# Experiementos com instrumentos par a par

### Combinação em par de categorias de instrumentos:

- aerofone e corda
- aerofone e sopro
- aerofone e percusão
- sopro e percusão
- sopro e corda
- percusão e corda

### Aerofone e corda

In [11]:
le = LabelEncoder()

df_aerofone_corda = df_cat_inst[df_cat_inst["cat_inst"].isin(["aerofone", "corda"])]
df_aerofone_corda["label_inst"] = le.fit_transform(df_aerofone_corda.instrumento)
df_aerofone_corda

,cat_inst,tonnetz0,tonnetz1,chroma1,chroma2,chroma3,chroma4,chroma5,chroma6,chroma7,...,spectral_band4,spectral_centroid,spectral_onset,spectral_rolloff,spectogram,mel_spectogram,zero_crossing_rate,zero_crossing,instrumento,label_inst
0,aerofone,0.010380,0.038218,0.513887,0.581963,0.434641,0.293949,0.317826,0.271148,0.548258,...,3409.640722,2600.055158,1.098243,5260.821321,-44.640682,-30.202150,0.149672,33095,accordion,0
1,aerofone,0.084643,-0.035337,0.175587,0.429345,0.840043,0.384524,0.282545,0.346341,0.482365,...,3516.595715,2385.271420,1.360742,5225.124166,-46.575737,-59.323132,0.112181,24785,accordion,0
2,aerofone,-0.030127,0.082404,0.453814,0.219808,0.228484,0.446050,0.178916,0.250981,0.327460,...,3256.491175,2069.850731,1.343197,4287.155824,-56.276707,-71.512909,0.096007,21230,accordion,0
3,aerofone,0.157274,-0.066099,0.209729,0.338755,0.660264,0.256625,0.275209,0.180417,0.399448,...,3855.791675,2903.979876,1.218963,6751.133809,-42.723160,-44.649315,0.144010,31836,accordion,0
4,aerofone,-0.049096,-0.029067,0.484400,0.334572,0.441629,0.679485,0.382958,0.326033,0.205775,...,2883.225952,1251.374287,1.339527,2263.034730,-62.227947,-78.196365,0.059804,13214,accordion,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8019,corda,0.051950,0.691044,0.773323,0.582712,0.532384,0.274606,0.127275,0.109617,0.158079,...,1906.686246,658.017342,1.227272,1367.932950,-35.302883,-68.164001,0.019043,4212,violin,9
8020,corda,0.015595,-0.058762,0.120028,0.238263,0.234643,0.267743,0.228591,0.260930,0.267262,...,3383.563150,3573.542978,1.354204,6552.813508,-43.596504,-73.498703,0.230540,50962,violin,9
8021,corda,0.093624,-0.031124,0.253249,0.321181,0.309254,0.265595,0.265015,0.155659,0.181344,...,3174.287104,1421.358194,1.173742,2751.578644,-48.512993,-66.277359,0.065123,14407,violin,9
8022,corda,0.004588,-0.009204,0.124913,0.174113,0.185558,0.169614,0.149416,0.075544,0.081894,...,3232.221409,3720.081212,1.332893,6207.732687,-37.720039,-72.552101,0.241853,53491,violin,9


- Tratar outilers

In [12]:
outilers_aerofone_corda  = ac.detected_outilers_knn(
    dataframe=df_aerofone_corda,
    algorithm='auto', 
    neighbors=5, 
    metric='euclidean'
)

In [13]:
df_aerofone_corda = ac.show_outilers(
    dataframe=df_aerofone_corda,
    pred=outilers_aerofone_corda['predictions']
)

df_aerofone_corda

,cat_inst,tonnetz0,tonnetz1,chroma1,chroma2,chroma3,chroma4,chroma5,chroma6,chroma7,...,spectral_band4,spectral_centroid,spectral_onset,spectral_rolloff,spectogram,mel_spectogram,zero_crossing_rate,zero_crossing,instrumento,label_inst
3,aerofone,0.157274,-0.066099,0.209729,0.338755,0.660264,0.256625,0.275209,0.180417,0.399448,...,3855.791675,2903.979876,1.218963,6751.133809,-42.723160,-44.649315,0.144010,31836,accordion,0
28,aerofone,0.177880,0.099179,0.446227,0.186154,0.310970,0.289374,0.480033,0.271526,0.262448,...,3006.696138,3178.625677,1.504104,5658.636007,-64.840324,-79.991013,0.225914,49936,accordion,0
46,aerofone,0.001389,0.058207,0.307783,0.359920,0.321503,0.340189,0.230577,0.230887,0.267941,...,3364.332940,3317.306522,1.480623,6072.413242,-37.332355,-79.453758,0.249911,55267,accordion,0
60,aerofone,0.003770,0.069227,0.157029,0.264191,0.612490,0.513482,0.283783,0.279843,0.223590,...,3543.097268,3223.649221,1.543456,6539.049245,-47.371098,-56.212872,0.160451,35507,accordion,0
68,aerofone,0.315369,0.119436,0.165054,0.304648,0.831039,0.422758,0.297391,0.241401,0.206774,...,3508.986076,2902.618765,1.674287,5889.156191,-42.290485,-77.100327,0.123588,27331,accordion,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7999,corda,0.188614,-0.123051,0.435805,0.078954,0.048469,0.095815,0.166382,0.088318,0.250419,...,2069.443752,1772.864514,1.268380,2945.977004,-76.595161,-80.000000,0.126178,27872,violin,9
8016,corda,0.066705,-0.239727,0.177657,0.249958,0.202065,0.227559,0.358576,0.191313,0.535777,...,3258.026601,2049.088272,1.239214,3472.241494,-45.379463,-80.000000,0.128462,28414,violin,9
8017,corda,0.096697,-0.279816,0.159033,0.078111,0.079078,0.180735,0.292732,0.125269,0.363708,...,3348.078759,2277.244488,1.186836,3779.127109,-50.537724,-80.000000,0.143759,31806,violin,9
8020,corda,0.015595,-0.058762,0.120028,0.238263,0.234643,0.267743,0.228591,0.260930,0.267262,...,3383.563150,3573.542978,1.354204,6552.813508,-43.596504,-73.498703,0.230540,50962,violin,9


- Treinamento de modelo

In [14]:
list_models = np.array([
    LGBMClassifier(),
    XGBClassifier(),
    GaussianNB(),
    KNeighborsClassifier(), 
    DecisionTreeClassifier(), 
    RandomForestClassifier(), 
    HistGradientBoostingClassifier(),
    MLPClassifier()
])

tm.train_models(
    X= df_aerofone_corda.drop(columns=['instrumento','label_inst', 'cat_inst']),
    y= df_aerofone_corda['label_inst'],
    models=list_models
)

Acuracia do modelo LGBMClassifier() do Fold 0: 0.4111111111111111
Acuracia do modelo LGBMClassifier() do Fold 1: 0.4888888888888889
Acuracia do modelo LGBMClassifier() do Fold 2: 0.28888888888888886
Acuracia do modelo LGBMClassifier() do Fold 3: 0.37777777777777777
Acuracia do modelo LGBMClassifier() do Fold 4: 0.38202247191011235
Acuracia do modelo XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_est

{'LGBMClassifier': 0.38973782771535576,
 'XGBClassifier(base_score=None, booster=None, callbacks=None,\n              colsample_bylevel=None, colsample_bynode=None,\n              colsample_bytree=None, early_stopping_rounds=None,\n              enable_categorical=False, eval_metric=None, feature_types=None,\n              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,\n              interaction_constraints=None, learning_rate=None, max_bin=None,\n              max_cat_threshold=None, max_cat_to_onehot=None,\n              max_delta_step=None, max_depth=None, max_leaves=None,\n              min_child_weight=None, missing=nan, monotone_constraints=None,\n              n_estimators=100, n_jobs=None, num_parallel_tree=None,\n              predictor=None, random_state=None, ..': 0.3931086142322097,
 'GaussianNB': 0.3756887224302954,
 'KNeighborsClassifier': 0.3463607990012484,
 'DecisionTreeClassifier': 0.32562796504369534,
 'RandomForestClassifier': 0.33522263836870586,


In [15]:
keys = le.classes_
values = le.transform(le.classes_)
dictionary = dict(zip(keys, values))
print(dictionary)

{'accordion': 0, 'banjo': 1, 'bass': 2, 'cello': 3, 'guitar': 4, 'mandolin': 5, 'organ': 6, 'piano': 7, 'ukulele': 8, 'violin': 9}


### Aerofone e sopro

In [22]:
df_aerofone_sopro = df_cat_inst[df_cat_inst["cat_inst"].isin(["aerofone", "sopro"])]
df_aerofone_sopro["label_inst"] = le.fit_transform(df_aerofone_sopro.instrumento)
df_aerofone_sopro

,cat_inst,tonnetz0,tonnetz1,chroma1,chroma2,chroma3,chroma4,chroma5,chroma6,chroma7,...,spectral_band4,spectral_centroid,spectral_onset,spectral_rolloff,spectogram,mel_spectogram,zero_crossing_rate,zero_crossing,instrumento,label_inst
0,aerofone,0.010380,0.038218,0.513887,0.581963,0.434641,0.293949,0.317826,0.271148,0.548258,...,3409.640722,2600.055158,1.098243,5260.821321,-44.640682,-30.202150,0.149672,33095,accordion,0
1,aerofone,0.084643,-0.035337,0.175587,0.429345,0.840043,0.384524,0.282545,0.346341,0.482365,...,3516.595715,2385.271420,1.360742,5225.124166,-46.575737,-59.323132,0.112181,24785,accordion,0
2,aerofone,-0.030127,0.082404,0.453814,0.219808,0.228484,0.446050,0.178916,0.250981,0.327460,...,3256.491175,2069.850731,1.343197,4287.155824,-56.276707,-71.512909,0.096007,21230,accordion,0
3,aerofone,0.157274,-0.066099,0.209729,0.338755,0.660264,0.256625,0.275209,0.180417,0.399448,...,3855.791675,2903.979876,1.218963,6751.133809,-42.723160,-44.649315,0.144010,31836,accordion,0
4,aerofone,-0.049096,-0.029067,0.484400,0.334572,0.441629,0.679485,0.382958,0.326033,0.205775,...,2883.225952,1251.374287,1.339527,2263.034730,-62.227947,-78.196365,0.059804,13214,accordion,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7161,sopro,0.031326,0.036042,0.281165,0.467478,0.383710,0.385898,0.358991,0.379168,0.286116,...,2040.836138,896.653459,1.242603,1782.859289,-42.632786,-42.830780,0.041767,9231,trumpet,7
7162,sopro,0.024069,-0.064162,0.038717,0.051984,0.156484,0.528343,0.865360,0.228249,0.063890,...,2528.740114,1280.731144,1.535133,1930.644010,-52.891537,-79.643761,0.078466,17360,trumpet,7
7163,sopro,0.046916,0.068088,0.517141,0.282860,0.547268,0.444383,0.516097,0.406917,0.322084,...,3643.088516,2309.708871,1.049440,5199.019529,-37.911655,-57.435429,0.095421,21114,trumpet,7
7164,sopro,0.109233,0.002179,0.271734,0.290186,0.454663,0.332584,0.266866,0.341721,0.481044,...,3440.696663,2370.998419,1.259253,5153.205266,-53.025776,-79.120621,0.126226,27933,trumpet,7


- Tratar outilers

In [23]:
outilers_aerofone_sopro  = ac.detected_outilers_knn(
    dataframe=df_aerofone_sopro,
    algorithm='auto', 
    neighbors=5, 
    metric='euclidean'
)

df_aerofone_sopro = ac.show_outilers(
    dataframe=df_aerofone_sopro,
    pred=outilers_aerofone_sopro['predictions']
)

df_aerofone_sopro

,cat_inst,tonnetz0,tonnetz1,chroma1,chroma2,chroma3,chroma4,chroma5,chroma6,chroma7,...,spectral_band4,spectral_centroid,spectral_onset,spectral_rolloff,spectogram,mel_spectogram,zero_crossing_rate,zero_crossing,instrumento,label_inst
28,aerofone,0.177880,0.099179,0.446227,0.186154,0.310970,0.289374,0.480033,0.271526,0.262448,...,3006.696138,3178.625677,1.504104,5658.636007,-64.840324,-79.991013,0.225914,49936,accordion,0
46,aerofone,0.001389,0.058207,0.307783,0.359920,0.321503,0.340189,0.230577,0.230887,0.267941,...,3364.332940,3317.306522,1.480623,6072.413242,-37.332355,-79.453758,0.249911,55267,accordion,0
60,aerofone,0.003770,0.069227,0.157029,0.264191,0.612490,0.513482,0.283783,0.279843,0.223590,...,3543.097268,3223.649221,1.543456,6539.049245,-47.371098,-56.212872,0.160451,35507,accordion,0
78,aerofone,0.092169,0.133666,0.193358,0.281352,0.774958,0.293572,0.316613,0.302946,0.372028,...,2537.886103,2105.938025,1.162251,3689.097337,-49.523079,-69.013954,0.146228,32336,accordion,0
93,aerofone,-0.002144,-0.008803,0.449169,0.438542,0.488127,0.524772,0.491166,0.454427,0.469509,...,3388.065392,3152.126942,1.361253,6306.830386,-32.494141,-51.824329,0.183488,40554,accordion,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7153,sopro,0.095131,0.165130,0.409808,0.178644,0.131329,0.086917,0.195207,0.238547,0.105908,...,1670.854795,1466.013189,1.140587,2337.326779,-75.481300,-79.999992,0.101459,22454,trumpet,7
7154,sopro,-0.029138,0.004838,0.534387,0.545401,0.223559,0.374719,0.498750,0.877884,0.388922,...,2662.124879,1776.745728,1.295782,3109.524430,-46.688988,-58.327690,0.127745,28264,trumpet,7
7156,sopro,-0.148693,0.003055,0.543909,0.820975,0.261659,0.262984,0.201507,0.296120,0.160154,...,2812.377753,2612.295812,0.958099,4009.572352,-64.094101,-80.000000,0.170468,37702,trumpet,7
7159,sopro,-0.040215,0.031151,0.587902,0.642199,0.661011,0.675489,0.697853,0.718583,0.482416,...,1626.501203,285.508118,0.854189,403.660127,-16.440603,-14.334520,0.014532,3213,trumpet,7


- Treinamento de Modelo

In [18]:
list_models = np.array([
    LGBMClassifier(),
    XGBClassifier(),
    GaussianNB(),
    KNeighborsClassifier(), 
    DecisionTreeClassifier(), 
    RandomForestClassifier(), 
    HistGradientBoostingClassifier(),
    MLPClassifier()
])

tm.train_models(
    X= df_aerofone_sopro.drop(columns=['instrumento','label_inst', 'cat_inst']),
    y= df_aerofone_sopro['label_inst'],
    models=list_models
)

Acuracia do modelo LGBMClassifier() do Fold 0: 0.5441176470588235
Acuracia do modelo LGBMClassifier() do Fold 1: 0.5223880597014925
Acuracia do modelo LGBMClassifier() do Fold 2: 0.5671641791044776
Acuracia do modelo LGBMClassifier() do Fold 3: 0.47761194029850745
Acuracia do modelo LGBMClassifier() do Fold 4: 0.5223880597014925
Acuracia do modelo XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estim

{'LGBMClassifier': 0.3433699425323263,
 'XGBClassifier(base_score=None, booster=None, callbacks=None,\n              colsample_bylevel=None, colsample_bynode=None,\n              colsample_bytree=None, early_stopping_rounds=None,\n              enable_categorical=False, eval_metric=None, feature_types=None,\n              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,\n              interaction_constraints=None, learning_rate=None, max_bin=None,\n              max_cat_threshold=None, max_cat_to_onehot=None,\n              max_delta_step=None, max_depth=None, max_leaves=None,\n              min_child_weight=None, missing=nan, monotone_constraints=None,\n              n_estimators=100, n_jobs=None, num_parallel_tree=None,\n              predictor=None, random_state=None, ..': 0.35963435302009444,
 'GaussianNB': 0.36075387348542387,
 'KNeighborsClassifier': 0.3617050981049807,
 'DecisionTreeClassifier': 0.3641139515701274,
 'RandomForestClassifier': 0.37424340153636504,


In [19]:
keys = le.classes_
values = le.transform(le.classes_)
dictionary = dict(zip(keys, values))
print(dictionary)

{'accordion': 0, 'clarinet': 1, 'flute': 2, 'organ': 3, 'piano': 4, 'saxophone': 5, 'trombone': 6, 'trumpet': 7}


### Aerofone e percusão

In [24]:
df_aerofone_percusao = df_cat_inst[df_cat_inst["cat_inst"].isin(["aerofone", "percusao"])]
df_aerofone_percusao["label_inst"] = le.fit_transform(df_aerofone_percusao.instrumento)
df_aerofone_percusao

,cat_inst,tonnetz0,tonnetz1,chroma1,chroma2,chroma3,chroma4,chroma5,chroma6,chroma7,...,spectral_band4,spectral_centroid,spectral_onset,spectral_rolloff,spectogram,mel_spectogram,zero_crossing_rate,zero_crossing,instrumento,label_inst
0,aerofone,0.010380,0.038218,0.513887,0.581963,0.434641,0.293949,0.317826,0.271148,0.548258,...,3409.640722,2600.055158,1.098243,5260.821321,-44.640682,-30.202150,0.149672,33095,accordion,0
1,aerofone,0.084643,-0.035337,0.175587,0.429345,0.840043,0.384524,0.282545,0.346341,0.482365,...,3516.595715,2385.271420,1.360742,5225.124166,-46.575737,-59.323132,0.112181,24785,accordion,0
2,aerofone,-0.030127,0.082404,0.453814,0.219808,0.228484,0.446050,0.178916,0.250981,0.327460,...,3256.491175,2069.850731,1.343197,4287.155824,-56.276707,-71.512909,0.096007,21230,accordion,0
3,aerofone,0.157274,-0.066099,0.209729,0.338755,0.660264,0.256625,0.275209,0.180417,0.399448,...,3855.791675,2903.979876,1.218963,6751.133809,-42.723160,-44.649315,0.144010,31836,accordion,0
4,aerofone,-0.049096,-0.029067,0.484400,0.334572,0.441629,0.679485,0.382958,0.326033,0.205775,...,2883.225952,1251.374287,1.339527,2263.034730,-62.227947,-78.196365,0.059804,13214,accordion,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4581,percusao,0.064085,0.031877,0.088719,0.110646,0.159631,0.303337,0.233713,0.293759,0.225420,...,2760.239016,1950.192940,1.101535,2585.158459,-50.985691,-72.046715,0.160147,35457,mallet_percussion,3
4582,percusao,0.264473,-0.245898,0.214395,0.094173,0.177360,0.087018,0.063969,0.138096,0.353633,...,3803.817212,1984.641346,1.996110,4595.240504,-66.942772,-78.150574,0.064426,14272,mallet_percussion,3
4583,percusao,0.087865,-0.048417,0.321572,0.221088,0.240128,0.374176,0.547956,0.418325,0.338949,...,3608.456357,3307.151107,1.378473,6980.305045,-56.667892,-75.132233,0.158234,35032,mallet_percussion,3
4584,percusao,0.432549,0.102409,0.204631,0.283638,0.785254,0.440283,0.356656,0.148266,0.208002,...,3119.488734,1533.762409,1.141370,2913.027706,-68.506294,-79.922241,0.079375,17537,mallet_percussion,3


- Tratar Outilers

In [25]:
outilers_aerofone_percusao  = ac.detected_outilers_knn(
    dataframe=df_aerofone_percusao,
    algorithm='auto', 
    neighbors=5, 
    metric='euclidean'
)

df_aerofone_percusao = ac.show_outilers(
    dataframe=df_aerofone_percusao,
    pred=outilers_aerofone_percusao['predictions']
)

df_aerofone_percusao

,cat_inst,tonnetz0,tonnetz1,chroma1,chroma2,chroma3,chroma4,chroma5,chroma6,chroma7,...,spectral_band4,spectral_centroid,spectral_onset,spectral_rolloff,spectogram,mel_spectogram,zero_crossing_rate,zero_crossing,instrumento,label_inst
28,aerofone,0.177880,0.099179,0.446227,0.186154,0.310970,0.289374,0.480033,0.271526,0.262448,...,3006.696138,3178.625677,1.504104,5658.636007,-64.840324,-79.991013,0.225914,49936,accordion,0
101,aerofone,-0.083369,0.002492,0.365614,0.188457,0.210712,0.340728,0.218881,0.389993,0.138017,...,2898.294706,2495.492458,1.521361,4334.344015,-64.275818,-79.999847,0.203175,44935,accordion,0
102,aerofone,0.304000,0.122806,0.350294,0.218089,0.486672,0.214657,0.331902,0.073321,0.085603,...,3110.390356,3408.975053,1.338741,6270.983349,-78.413834,-80.000000,0.231030,51151,accordion,0
135,aerofone,0.023928,0.295064,0.578289,0.594822,0.568853,0.723615,0.453601,0.691517,0.612611,...,3287.098423,788.124857,0.801734,1317.872000,-0.801849,-2.722743,0.004142,966,accordion,0
137,aerofone,0.024385,0.039629,0.464126,0.419017,0.569746,0.737048,0.675182,0.382305,0.338771,...,4328.827405,4383.260067,0.988195,8659.220415,-55.435081,-64.208626,0.376119,83194,accordion,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4561,percusao,0.305108,-0.047513,0.147523,0.121128,0.047512,0.028472,0.247569,0.205764,0.088979,...,3832.494503,2511.562704,1.421833,5474.379736,-47.792496,-70.003914,0.146273,32362,mallet_percussion,3
4573,percusao,0.424326,-0.240209,0.127338,0.211656,0.438093,0.604280,0.843284,0.976121,0.586622,...,1068.042998,283.995303,0.419410,510.951435,-62.118206,-73.764893,0.012129,2688,mallet_percussion,3
4577,percusao,0.348110,0.159918,0.278293,0.092337,0.303201,0.065521,0.185706,0.234855,0.103411,...,2891.545646,2264.087534,1.359796,3551.354782,-21.233885,-59.256969,0.166638,36868,mallet_percussion,3
4581,percusao,0.064085,0.031877,0.088719,0.110646,0.159631,0.303337,0.233713,0.293759,0.225420,...,2760.239016,1950.192940,1.101535,2585.158459,-50.985691,-72.046715,0.160147,35457,mallet_percussion,3


- Treinamento de Modelo

In [26]:
list_models = np.array([
    LGBMClassifier(),
    XGBClassifier(),
    GaussianNB(),
    KNeighborsClassifier(), 
    DecisionTreeClassifier(), 
    RandomForestClassifier(), 
    HistGradientBoostingClassifier(),
    MLPClassifier()
])

tm.train_models(
    X= df_aerofone_percusao.drop(columns=['instrumento','label_inst', 'cat_inst']),
    y= df_aerofone_percusao['label_inst'],
    models=list_models
)

Acuracia do modelo LGBMClassifier() do Fold 0: 0.5373134328358209
Acuracia do modelo LGBMClassifier() do Fold 1: 0.4696969696969697
Acuracia do modelo LGBMClassifier() do Fold 2: 0.5
Acuracia do modelo LGBMClassifier() do Fold 3: 0.6060606060606061
Acuracia do modelo LGBMClassifier() do Fold 4: 0.5454545454545454
Acuracia do modelo XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_job

{'LGBMClassifier': 0.38551347426075494,
 'XGBClassifier(base_score=None, booster=None, callbacks=None,\n              colsample_bylevel=None, colsample_bynode=None,\n              colsample_bytree=None, early_stopping_rounds=None,\n              enable_categorical=False, eval_metric=None, feature_types=None,\n              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,\n              interaction_constraints=None, learning_rate=None, max_bin=None,\n              max_cat_threshold=None, max_cat_to_onehot=None,\n              max_delta_step=None, max_depth=None, max_leaves=None,\n              min_child_weight=None, missing=nan, monotone_constraints=None,\n              n_estimators=100, n_jobs=None, num_parallel_tree=None,\n              predictor=None, random_state=None, ..': 0.3931402320980702,
 'GaussianNB': 0.3958222037429836,
 'KNeighborsClassifier': 0.39325633100495244,
 'DecisionTreeClassifier': 0.39265996763022537,
 'RandomForestClassifier': 0.3991199982944574,


In [27]:
keys = le.classes_
values = le.transform(le.classes_)
dictionary = dict(zip(keys, values))
print(dictionary)

{'accordion': 0, 'cymbals': 1, 'drums': 2, 'mallet_percussion': 3, 'organ': 4, 'piano': 5}


### Sopro e Percusão

In [28]:
df_sopro_percusao = df_cat_inst[df_cat_inst["cat_inst"].isin(["sopro", "percusao"])]
df_sopro_percusao["label_inst"] = le.fit_transform(df_sopro_percusao.instrumento)
df_sopro_percusao

,cat_inst,tonnetz0,tonnetz1,chroma1,chroma2,chroma3,chroma4,chroma5,chroma6,chroma7,...,spectral_band4,spectral_centroid,spectral_onset,spectral_rolloff,spectogram,mel_spectogram,zero_crossing_rate,zero_crossing,instrumento,label_inst
2027,percusao,0.040087,0.116203,0.409334,0.260820,0.298635,0.223068,0.472596,0.682380,0.315802,...,3634.193997,2843.107702,0.915473,6288.444728,-72.321693,-79.942192,0.106036,23418,cymbals,1
2028,percusao,0.064332,-0.023445,0.221986,0.318158,0.420399,0.800723,0.699277,0.292424,0.310495,...,2783.919958,2549.517812,1.755609,4183.087002,-47.144733,-65.961197,0.182486,40358,cymbals,1
2029,percusao,-0.039875,0.021506,0.597219,0.522248,0.357256,0.306705,0.454092,0.676822,0.459109,...,3093.350146,2543.370274,0.869033,4545.004690,-59.076328,-70.129898,0.152626,33743,cymbals,1
2030,percusao,-0.039822,0.028304,0.621655,0.511336,0.348413,0.316168,0.392748,0.568508,0.565586,...,3709.821567,2479.813426,1.396642,5702.476809,-63.114140,-71.209488,0.082574,18250,cymbals,1
2031,percusao,0.098013,0.027428,0.383891,0.186740,0.131834,0.186899,0.345623,0.210410,0.242533,...,3467.632804,2037.566496,1.731518,4412.483063,-56.386051,-59.750889,0.079232,17508,cymbals,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7161,sopro,0.031326,0.036042,0.281165,0.467478,0.383710,0.385898,0.358991,0.379168,0.286116,...,2040.836138,896.653459,1.242603,1782.859289,-42.632786,-42.830780,0.041767,9231,trumpet,7
7162,sopro,0.024069,-0.064162,0.038717,0.051984,0.156484,0.528343,0.865360,0.228249,0.063890,...,2528.740114,1280.731144,1.535133,1930.644010,-52.891537,-79.643761,0.078466,17360,trumpet,7
7163,sopro,0.046916,0.068088,0.517141,0.282860,0.547268,0.444383,0.516097,0.406917,0.322084,...,3643.088516,2309.708871,1.049440,5199.019529,-37.911655,-57.435429,0.095421,21114,trumpet,7
7164,sopro,0.109233,0.002179,0.271734,0.290186,0.454663,0.332584,0.266866,0.341721,0.481044,...,3440.696663,2370.998419,1.259253,5153.205266,-53.025776,-79.120621,0.126226,27933,trumpet,7


- Tratar outilers

In [29]:
outilers_sopro_percusao  = ac.detected_outilers_knn(
    dataframe=df_sopro_percusao,
    algorithm='auto', 
    neighbors=5, 
    metric='euclidean'
)

df_sopro_percusao = ac.show_outilers(
    dataframe=df_sopro_percusao,
    pred=outilers_sopro_percusao['predictions']
)

df_sopro_percusao

,cat_inst,tonnetz0,tonnetz1,chroma1,chroma2,chroma3,chroma4,chroma5,chroma6,chroma7,...,spectral_band4,spectral_centroid,spectral_onset,spectral_rolloff,spectogram,mel_spectogram,zero_crossing_rate,zero_crossing,instrumento,label_inst
2058,percusao,0.028724,0.119361,0.387632,0.360637,0.641758,0.430467,0.488429,0.566602,0.417376,...,3690.605020,3368.580434,1.101028,7120.545651,-64.049034,-75.085358,0.179711,39714,cymbals,1
2066,percusao,0.067263,-0.079658,0.400289,0.414243,0.519233,0.403937,0.590723,0.326989,0.475230,...,3505.349304,3813.025950,1.273033,7134.060110,-47.323631,-62.697948,0.257473,56977,cymbals,1
2080,percusao,-0.028268,0.020594,0.308341,0.539134,0.486185,0.370537,0.365475,0.277279,0.237507,...,3470.966019,5631.779972,1.150056,8727.017531,-66.101967,-79.368797,0.486816,107670,cymbals,1
2084,percusao,0.090227,0.001803,0.181475,0.215828,0.223079,0.230837,0.477740,0.319292,0.496053,...,2945.350082,2887.785890,1.069310,5327.569254,-51.677456,-71.216888,0.188245,41621,cymbals,1
2087,percusao,0.000423,0.008728,0.315313,0.329236,0.318093,0.291647,0.358462,0.355331,0.340839,...,3459.666432,4034.598255,2.311115,7370.300831,-48.193481,-79.993744,0.275738,60975,cymbals,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7049,sopro,0.299879,0.026765,0.283813,0.178020,0.295542,0.277703,0.780676,0.233810,0.084910,...,1289.474969,985.032667,0.765431,1431.758164,-79.855415,-80.000000,0.066409,14678,trombone,6
7111,sopro,0.135470,0.111472,0.518805,0.231038,0.153544,0.142523,0.404127,0.336685,0.211471,...,1839.820221,1345.818619,0.946602,1966.715872,-78.892021,-80.000000,0.112739,24948,trumpet,7
7113,sopro,0.041526,0.006235,0.385098,0.320545,0.374398,0.359685,0.384015,0.473004,0.283592,...,1107.607570,1730.927792,1.009361,2816.952649,-76.458954,-80.000000,0.162672,35979,trumpet,7
7153,sopro,0.095131,0.165130,0.409808,0.178644,0.131329,0.086917,0.195207,0.238547,0.105908,...,1670.854795,1466.013189,1.140587,2337.326779,-75.481300,-79.999992,0.101459,22454,trumpet,7


- Treinamento de Modelo

In [30]:
list_models = np.array([
    LGBMClassifier(),
    XGBClassifier(),
    GaussianNB(),
    KNeighborsClassifier(), 
    DecisionTreeClassifier(), 
    RandomForestClassifier(), 
    HistGradientBoostingClassifier(),
    MLPClassifier()
])

tm.train_models(
    X= df_sopro_percusao.drop(columns=['instrumento','label_inst', 'cat_inst']),
    y= df_sopro_percusao['label_inst'],
    models=list_models
)

Acuracia do modelo LGBMClassifier() do Fold 0: 0.453125
Acuracia do modelo LGBMClassifier() do Fold 1: 0.40625
Acuracia do modelo LGBMClassifier() do Fold 2: 0.4603174603174603
Acuracia do modelo LGBMClassifier() do Fold 3: 0.47619047619047616
Acuracia do modelo LGBMClassifier() do Fold 4: 0.3968253968253968
Acuracia do modelo XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=Non

{'LGBMClassifier': 0.4010422538586883,
 'XGBClassifier(base_score=None, booster=None, callbacks=None,\n              colsample_bylevel=None, colsample_bynode=None,\n              colsample_bytree=None, early_stopping_rounds=None,\n              enable_categorical=False, eval_metric=None, feature_types=None,\n              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,\n              interaction_constraints=None, learning_rate=None, max_bin=None,\n              max_cat_threshold=None, max_cat_to_onehot=None,\n              max_delta_step=None, max_depth=None, max_leaves=None,\n              min_child_weight=None, missing=nan, monotone_constraints=None,\n              n_estimators=100, n_jobs=None, num_parallel_tree=None,\n              predictor=None, random_state=None, ..': 0.4039402013598498,
 'GaussianNB': 0.402181246224241,
 'KNeighborsClassifier': 0.3989765730314252,
 'DecisionTreeClassifier': 0.39763152699914284,
 'RandomForestClassifier': 0.4015150395912349,
 'H

In [31]:
keys = le.classes_
values = le.transform(le.classes_)
dictionary = dict(zip(keys, values))
print(dictionary)

{'clarinet': 0, 'cymbals': 1, 'drums': 2, 'flute': 3, 'mallet_percussion': 4, 'saxophone': 5, 'trombone': 6, 'trumpet': 7}


### Sopro e corda

In [32]:
df_sopro_corda = df_cat_inst[df_cat_inst["cat_inst"].isin(["sopro", "corda"])]
df_sopro_corda["label_inst"] = le.fit_transform(df_sopro_corda.instrumento)


- Tratar Outilers

In [33]:
outilers_sopro_corda  = ac.detected_outilers_knn(
    dataframe=df_sopro_corda,
    algorithm='auto', 
    neighbors=5, 
    metric='euclidean'
)

df_sopro_corda = ac.show_outilers(
    dataframe=df_sopro_corda,
    pred=outilers_sopro_corda['predictions']
)

df_sopro_corda

,cat_inst,tonnetz0,tonnetz1,chroma1,chroma2,chroma3,chroma4,chroma5,chroma6,chroma7,...,spectral_band4,spectral_centroid,spectral_onset,spectral_rolloff,spectogram,mel_spectogram,zero_crossing_rate,zero_crossing,instrumento,label_inst
1876,sopro,-0.044667,0.260524,0.692127,0.238955,0.029413,0.424056,0.732275,0.106237,0.264656,...,2269.854161,1872.058375,0.862944,2738.513835,-70.011116,-80.000000,0.139929,30932,clarinet,3
1878,sopro,0.089747,0.253008,0.299878,0.225017,0.544357,0.164941,0.079692,0.119684,0.194826,...,2431.808142,1999.856294,1.274141,3282.789275,-58.643265,-71.021973,0.176906,39102,clarinet,3
1886,sopro,0.002779,0.015584,0.311722,0.319367,0.339069,0.368589,0.345589,0.443560,0.344975,...,2937.446277,3154.871456,1.945904,5838.245903,-44.260166,-62.325306,0.232276,51374,clarinet,3
1887,sopro,0.105165,0.011241,0.407080,0.577214,0.610102,0.431792,0.194139,0.107201,0.066060,...,3682.947087,2288.592275,0.987397,5162.173271,-58.093662,-77.751862,0.046046,10185,clarinet,3
1888,sopro,0.108679,0.066236,0.484411,0.352953,0.433525,0.316416,0.265250,0.316138,0.292531,...,3654.260611,1992.119222,1.766695,4726.313261,-42.178364,-41.680836,0.041709,9216,clarinet,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7999,corda,0.188614,-0.123051,0.435805,0.078954,0.048469,0.095815,0.166382,0.088318,0.250419,...,2069.443752,1772.864514,1.268380,2945.977004,-76.595161,-80.000000,0.126178,27872,violin,11
8016,corda,0.066705,-0.239727,0.177657,0.249958,0.202065,0.227559,0.358576,0.191313,0.535777,...,3258.026601,2049.088272,1.239214,3472.241494,-45.379463,-80.000000,0.128462,28414,violin,11
8017,corda,0.096697,-0.279816,0.159033,0.078111,0.079078,0.180735,0.292732,0.125269,0.363708,...,3348.078759,2277.244488,1.186836,3779.127109,-50.537724,-80.000000,0.143759,31806,violin,11
8020,corda,0.015595,-0.058762,0.120028,0.238263,0.234643,0.267743,0.228591,0.260930,0.267262,...,3383.563150,3573.542978,1.354204,6552.813508,-43.596504,-73.498703,0.230540,50962,violin,11


- Treinamento de Modelo

In [34]:
list_models = np.array([
    LGBMClassifier(),
    XGBClassifier(),
    GaussianNB(),
    KNeighborsClassifier(), 
    DecisionTreeClassifier(), 
    RandomForestClassifier(), 
    HistGradientBoostingClassifier(),
    MLPClassifier()
])

tm.train_models(
    X= df_sopro_corda.drop(columns=['instrumento','label_inst', 'cat_inst']),
    y= df_sopro_corda['label_inst'],
    models=list_models
)

Acuracia do modelo LGBMClassifier() do Fold 0: 0.41379310344827586
Acuracia do modelo LGBMClassifier() do Fold 1: 0.4827586206896552
Acuracia do modelo LGBMClassifier() do Fold 2: 0.45977011494252873
Acuracia do modelo LGBMClassifier() do Fold 3: 0.4942528735632184
Acuracia do modelo LGBMClassifier() do Fold 4: 0.43023255813953487
Acuracia do modelo XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_est

{'LGBMClassifier': 0.40130247543978,
 'XGBClassifier(base_score=None, booster=None, callbacks=None,\n              colsample_bylevel=None, colsample_bynode=None,\n              colsample_bytree=None, early_stopping_rounds=None,\n              enable_categorical=False, eval_metric=None, feature_types=None,\n              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,\n              interaction_constraints=None, learning_rate=None, max_bin=None,\n              max_cat_threshold=None, max_cat_to_onehot=None,\n              max_delta_step=None, max_depth=None, max_leaves=None,\n              min_child_weight=None, missing=nan, monotone_constraints=None,\n              n_estimators=100, n_jobs=None, num_parallel_tree=None,\n              predictor=None, random_state=None, ..': 0.4028452167591801,
 'GaussianNB': 0.4012112460416784,
 'KNeighborsClassifier': 0.3988327133302679,
 'DecisionTreeClassifier': 0.396155779279042,
 'RandomForestClassifier': 0.3979156607469657,
 'Hist

In [35]:
keys = le.classes_
values = le.transform(le.classes_)
dictionary = dict(zip(keys, values))
print(dictionary)

{'banjo': 0, 'bass': 1, 'cello': 2, 'clarinet': 3, 'flute': 4, 'guitar': 5, 'mandolin': 6, 'saxophone': 7, 'trombone': 8, 'trumpet': 9, 'ukulele': 10, 'violin': 11}


### Corda e Percusão

In [36]:
df_corda_percusao = df_cat_inst[df_cat_inst["cat_inst"].isin(["corda", "percusao"])]
df_corda_percusao["label_inst"] = le.fit_transform(df_corda_percusao.instrumento)
df_corda_percusao

,cat_inst,tonnetz0,tonnetz1,chroma1,chroma2,chroma3,chroma4,chroma5,chroma6,chroma7,...,spectral_band4,spectral_centroid,spectral_onset,spectral_rolloff,spectogram,mel_spectogram,zero_crossing_rate,zero_crossing,instrumento,label_inst
2027,percusao,0.040087,0.116203,0.409334,0.260820,0.298635,0.223068,0.472596,0.682380,0.315802,...,3634.193997,2843.107702,0.915473,6288.444728,-72.321693,-79.942192,0.106036,23418,cymbals,3
2028,percusao,0.064332,-0.023445,0.221986,0.318158,0.420399,0.800723,0.699277,0.292424,0.310495,...,2783.919958,2549.517812,1.755609,4183.087002,-47.144733,-65.961197,0.182486,40358,cymbals,3
2029,percusao,-0.039875,0.021506,0.597219,0.522248,0.357256,0.306705,0.454092,0.676822,0.459109,...,3093.350146,2543.370274,0.869033,4545.004690,-59.076328,-70.129898,0.152626,33743,cymbals,3
2030,percusao,-0.039822,0.028304,0.621655,0.511336,0.348413,0.316168,0.392748,0.568508,0.565586,...,3709.821567,2479.813426,1.396642,5702.476809,-63.114140,-71.209488,0.082574,18250,cymbals,3
2031,percusao,0.098013,0.027428,0.383891,0.186740,0.131834,0.186899,0.345623,0.210410,0.242533,...,3467.632804,2037.566496,1.731518,4412.483063,-56.386051,-59.750889,0.079232,17508,cymbals,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8019,corda,0.051950,0.691044,0.773323,0.582712,0.532384,0.274606,0.127275,0.109617,0.158079,...,1906.686246,658.017342,1.227272,1367.932950,-35.302883,-68.164001,0.019043,4212,violin,9
8020,corda,0.015595,-0.058762,0.120028,0.238263,0.234643,0.267743,0.228591,0.260930,0.267262,...,3383.563150,3573.542978,1.354204,6552.813508,-43.596504,-73.498703,0.230540,50962,violin,9
8021,corda,0.093624,-0.031124,0.253249,0.321181,0.309254,0.265595,0.265015,0.155659,0.181344,...,3174.287104,1421.358194,1.173742,2751.578644,-48.512993,-66.277359,0.065123,14407,violin,9
8022,corda,0.004588,-0.009204,0.124913,0.174113,0.185558,0.169614,0.149416,0.075544,0.081894,...,3232.221409,3720.081212,1.332893,6207.732687,-37.720039,-72.552101,0.241853,53491,violin,9


- Tratamento Outilers

In [37]:
outilers_sopro_percusao  = ac.detected_outilers_knn(
    dataframe=df_sopro_percusao,
    algorithm='auto', 
    neighbors=5, 
    metric='euclidean'
)

df_sopro_percusao = ac.show_outilers(
    dataframe=df_sopro_percusao,
    pred=outilers_sopro_percusao['predictions']
)

df_sopro_percusao

,cat_inst,tonnetz0,tonnetz1,chroma1,chroma2,chroma3,chroma4,chroma5,chroma6,chroma7,...,spectral_band4,spectral_centroid,spectral_onset,spectral_rolloff,spectogram,mel_spectogram,zero_crossing_rate,zero_crossing,instrumento,label_inst
2101,percusao,-0.064002,-0.082437,0.295148,0.560085,0.600610,0.642566,0.403012,0.443036,0.612245,...,3276.475718,5693.551293,1.457964,8663.267259,-45.137260,-49.054279,0.473339,104617,cymbals,1
2173,percusao,0.101910,0.062642,0.614649,0.531424,0.419265,0.321603,0.317893,0.201710,0.157118,...,3105.281335,5434.601679,0.793554,8471.791618,-77.384094,-80.000000,0.464703,102828,cymbals,1
2298,percusao,-0.037618,-0.010607,0.526870,0.577688,0.628836,0.732237,0.697772,0.558642,0.428523,...,3762.965569,5452.680027,1.299207,8818.021543,-53.556351,-78.776329,0.434971,96225,cymbals,1
2300,percusao,-0.014834,0.006340,0.608367,0.624544,0.693369,0.593242,0.489754,0.660825,0.550813,...,3380.454092,5593.320227,1.205462,8877.100459,-26.379168,-56.951633,0.488289,107951,cymbals,1
2506,percusao,0.049372,0.234583,0.280210,0.249857,0.313773,0.393309,0.467969,0.481419,0.581691,...,3898.254426,1400.801263,0.536122,2024.645685,-47.109444,-36.347408,0.056344,12461,cymbals,1
2587,percusao,0.005174,0.011557,0.682791,0.699664,0.633348,0.558333,0.466065,0.486025,0.545153,...,3869.967172,5056.103646,1.250136,8604.613012,-57.817997,-60.998730,0.354021,78190,drums,2
2593,percusao,-0.014498,0.009951,0.355126,0.241228,0.208534,0.249382,0.323961,0.212346,0.206693,...,3515.335042,5613.715026,0.901109,8828.863086,-28.670713,-49.088226,0.595252,131659,drums,2
2603,percusao,0.004651,0.007252,0.420967,0.451960,0.514814,0.422149,0.370766,0.410011,0.610613,...,3605.427847,5109.887076,2.731347,8599.317143,-49.728470,-65.893913,0.420253,92913,drums,2
2681,percusao,-0.007345,-0.002354,0.858680,0.784684,0.776198,0.637403,0.574642,0.574306,0.585518,...,3614.312471,4917.856990,1.187007,8741.256424,-31.044067,-38.034096,0.486983,107709,drums,2
2708,percusao,-0.002698,-0.016959,0.706164,0.681776,0.664909,0.715067,0.726105,0.628962,0.582693,...,3509.751660,4969.901117,1.212538,8594.046254,-35.268333,-41.843285,0.384941,85150,drums,2


- Treinamento de Modelos

In [38]:
list_models = np.array([
    LGBMClassifier(),
    XGBClassifier(),
    GaussianNB(),
    KNeighborsClassifier(), 
    DecisionTreeClassifier(), 
    RandomForestClassifier(), 
    HistGradientBoostingClassifier(),
    MLPClassifier()
])

tm.train_models(
    X= df_sopro_corda.drop(columns=['instrumento','label_inst', 'cat_inst']),
    y= df_sopro_corda['label_inst'],
    models=list_models
)

Acuracia do modelo LGBMClassifier() do Fold 0: 0.41379310344827586
Acuracia do modelo LGBMClassifier() do Fold 1: 0.4827586206896552
Acuracia do modelo LGBMClassifier() do Fold 2: 0.45977011494252873
Acuracia do modelo LGBMClassifier() do Fold 3: 0.4942528735632184
Acuracia do modelo LGBMClassifier() do Fold 4: 0.43023255813953487
Acuracia do modelo XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_est

{'LGBMClassifier': 0.39504367881593655,
 'XGBClassifier(base_score=None, booster=None, callbacks=None,\n              colsample_bylevel=None, colsample_bynode=None,\n              colsample_bytree=None, early_stopping_rounds=None,\n              enable_categorical=False, eval_metric=None, feature_types=None,\n              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,\n              interaction_constraints=None, learning_rate=None, max_bin=None,\n              max_cat_threshold=None, max_cat_to_onehot=None,\n              max_delta_step=None, max_depth=None, max_leaves=None,\n              min_child_weight=None, missing=nan, monotone_constraints=None,\n              n_estimators=100, n_jobs=None, num_parallel_tree=None,\n              predictor=None, random_state=None, ..': 0.39644158361316145,
 'GaussianNB': 0.39526052914882326,
 'KNeighborsClassifier': 0.39344970049614314,
 'DecisionTreeClassifier': 0.39157203659429834,
 'RandomForestClassifier': 0.3932225428010829